In [1]:
import pandas as pd
 
# File paths
draft_classes_path = 'data/Draft Classes 2014-2023.csv'
epm_data_path = 'data/EPM 2014-2024.csv'

# Load the datasets
draft_classes = pd.read_csv(draft_classes_path)
epm_data = pd.read_csv(epm_data_path)

# Clean player names to ensure consistency between datasets
draft_classes['Player Name'] = draft_classes['Player Name'].str.strip().str.lower()
epm_data['Player Name'] = epm_data['Player Name'].str.strip().str.lower()

# Remove any columns that are entirely empty (all values NaN)
draft_classes = draft_classes.dropna(axis=1, how='all')


In [2]:
# Function to check eligibility based on minutes played
def is_eligible(minutes_series, required_years):
    return (minutes_series > 0).sum() >= required_years

# Function to extract player stats from the EPM dataset for each year
def get_player_stats(player_name, epm_data, year):
    player_stats = epm_data[(epm_data['Player Name'] == player_name) & (epm_data['Season'] == year)]
    return player_stats

In [4]:
import unicodedata
import re

# Function to normalize player names
def normalize_player_name(name):
    # Normalize non-English characters
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
    
    # Remove periods and extra spaces
    name = re.sub(r'\s+', ' ', name)  # Replace multiple spaces with a single space
    name = name.replace('.', '').strip()  # Remove periods
    
    # Normalize Roman numerals
    name = re.sub(r'\s*(i{1,3}|iv|vi{0,3}|v?i{0,3})$', '', name)  # Remove trailing Roman numerals
    
    return name

# Normalize player names in the Draft Classes dataset
draft_classes['Normalized Player Name'] = draft_classes['Player Name'].apply(normalize_player_name)

# Create a mapping from the EPM data
epm_data['Normalized Player Name'] = epm_data['Player Name'].apply(normalize_player_name)
name_mapping = dict(zip(epm_data['Normalized Player Name'], epm_data['Player Name']))

# Update the Draft Classes dataset with names from the EPM dataset
draft_classes['Player Name'] = draft_classes['Normalized Player Name'].map(name_mapping)

# Clean up
draft_classes.drop(columns=['Normalized Player Name'], inplace=True)

In [5]:
# Loop through each player in the entire draft class dataset and populate EPM and minutes for each year
for index, row in draft_classes.iterrows():
    player_name = row['Player Name']  # matching player names

    for year in range(2014, 2025):  # Loop through available years
        player_stats = get_player_stats(player_name, epm_data, year)
        
        minutes_col = f'Minutes_{year}'
        epm_col = f'EPM_{year}'
        cumulative_epm_col = f'Cumulative_EPM_{year}'
        
        # Initialize minutes and EPM with zeros
        draft_classes.loc[index, minutes_col] = 0  # Set default to 0
        draft_classes.loc[index, epm_col] = 0  # Set default to 0
        draft_classes.loc[index, cumulative_epm_col] = 0

        # If player data exists for that year, extract and assign the minutes and EPM
        if not player_stats.empty:
            draft_classes.loc[index, minutes_col] = player_stats['Minutes'].values[0]  # Total minutes played
            draft_classes.loc[index, epm_col] = player_stats['EPM'].values[0]  # EPM value

        # Calculate cumulative EPM (EPM * Minutes), only if minutes are greater than 0
        if draft_classes.loc[index, minutes_col] > 0:
            draft_classes.loc[index, cumulative_epm_col] = (draft_classes.loc[index, epm_col] * 
                                                             draft_classes.loc[index, minutes_col])

In [6]:
# Calculate cumulative EPM for the entire career, first 5 years, and best 3 years
for index, row in draft_classes.iterrows():
    draft_year = row['Draft Year']  # matching player names

    # Get all EPM and Minutes values for the player
    epm_values = row[[f'Cumulative_EPM_{year}' for year in range(2014, 2025)]].values
    minutes_values = row[[f'Minutes_{year}' for year in range(2014, 2025)]].values
    
    # Calculate the total career cumulative EPM (including only years with minutes > 0)
    total_cumulative_epm = sum(epm_values[minutes_values > 0])
    draft_classes.loc[index, 'Total_Cumulative_EPM'] = total_cumulative_epm
    
    # Calculate cumulative EPM for the first 5 years after the draft year
    start_index = draft_year - 2014 + 1  # Find the start index for years after the draft
    first_5_years_cumulative_epm = sum(epm_values[start_index:start_index + 5][minutes_values[start_index:start_index + 5] > 0])
    draft_classes.loc[index, 'First_5_Years_Cumulative_EPM'] = first_5_years_cumulative_epm
    
    # Store minutes played in the first 5 years
    first_5_years_minutes = sum(minutes_values[start_index:start_index + 5][minutes_values[start_index:start_index + 5] > 0])
    draft_classes.loc[index, 'First_5_Years_Minutes'] = first_5_years_minutes
    
    
    # Calculate cumulative EPM for the best 3 years
    # Filter out the years where the player has minutes > 0
    valid_years = [year for year in range(2014, 2025) if row[f'Minutes_{year}'] > 0]

    # If the player has fewer than 3 valid years, only use available years
    if len(valid_years) < 3:
        # If less than 3 valid years with minutes > 0, set values to 0
        draft_classes.loc[index, 'Best_3_Years_Cumulative_EPM'] = 0
        draft_classes.loc[index, 'Best_3_Years_Minutes'] = 0
    else:
        valid_epm_minutes = [(row[f'EPM_{year}'], row[f'Minutes_{year}']) for year in valid_years]

        # Sort the list of tuples (EPM, Minutes) by EPM in descending order
        sorted_epm_minutes = sorted(valid_epm_minutes, key=lambda x: x[0], reverse=True)

        # Select the best 3 years (or fewer if not enough years)
        best_3_years = sorted_epm_minutes[:3]

        # Calculate cumulative EPM for the best 3 years
        best_3_years_cumulative_epm = sum(epm * minutes for epm, minutes in best_3_years)
        draft_classes.loc[index, 'Best_3_Years_Cumulative_EPM'] = best_3_years_cumulative_epm

        # Store minutes played in the best 3 years
        best_3_years_minutes = sum(minutes for _, minutes in best_3_years)
        draft_classes.loc[index, 'Best_3_Years_Minutes'] = best_3_years_minutes


In [7]:
# Create rankings within each draft class for total, first 5 years, and best 3 years
for year in range(2014, 2025):
    draft_classes['Rank_Cumulative_EPM'] = draft_classes.groupby('Draft Year')['Total_Cumulative_EPM'].rank(method='min', ascending=False)
    draft_classes['Rank_First_5_Years_EPM'] = draft_classes.groupby('Draft Year')['First_5_Years_Cumulative_EPM'].rank(method='min', ascending=False)
    draft_classes['Rank_Best_3_Years_EPM'] = draft_classes.groupby('Draft Year')['Best_3_Years_Cumulative_EPM'].rank(method='min', ascending=False)

# Set rank to the lowest for players with zero minutes
for index, row in draft_classes.iterrows():
    if row['Total_Cumulative_EPM'] == 0:
        draft_classes.loc[index, 'Rank_Cumulative_EPM'] = draft_classes['Rank_Cumulative_EPM'].max() + 1
    if row['First_5_Years_Cumulative_EPM'] == 0:
        draft_classes.loc[index, 'Rank_First_5_Years_EPM'] = draft_classes['Rank_First_5_Years_EPM'].max() + 1
    if row['Best_3_Years_Cumulative_EPM'] == 0:
        draft_classes.loc[index, 'Rank_Best_3_Years_EPM'] = draft_classes['Rank_Best_3_Years_EPM'].max() + 1

In [8]:
# Calculate percentiles for ranks
draft_classes['Percentile_Cumulative_EPM'] = draft_classes.apply(
    lambda x: (1 - (x['Rank_Cumulative_EPM'] - 1) / draft_classes[draft_classes['Draft Year'] == x['Draft Year']].shape[0]) * 100,
    axis=1
)

draft_classes['Percentile_First_5_Years_EPM'] = draft_classes.apply(
    lambda x: (1 - (x['Rank_First_5_Years_EPM'] - 1) / draft_classes[draft_classes['Draft Year'] == x['Draft Year']].shape[0]) * 100,
    axis=1
)

draft_classes['Percentile_Best_3_Years_EPM'] = draft_classes.apply(
    lambda x: (1 - (x['Rank_Best_3_Years_EPM'] - 1) / draft_classes[draft_classes['Draft Year'] == x['Draft Year']].shape[0]) * 100,
    axis=1
)

In [9]:
# Reorder columns to place cumulative EPM and percentiles together
new_column_order = []
for year in range(2014, 2025):
    new_column_order.append(f'Minutes_{year}')
    new_column_order.append(f'EPM_{year}')
    new_column_order.append(f'Cumulative_EPM_{year}')

# Get columns that are not part of the new order
other_columns = draft_classes.columns.difference(new_column_order).tolist()
final_column_order = other_columns + new_column_order + [
    'Total_Cumulative_EPM', 
    'Rank_Cumulative_EPM',
    'First_5_Years_Cumulative_EPM',
    'Rank_First_5_Years_EPM',
    'Best_3_Years_Cumulative_EPM',
    'Rank_Best_3_Years_EPM'
]

# Reindex the DataFrame to the new order
draft_classes = draft_classes[final_column_order]

In [10]:
draft_classes

,Best_3_Years_Cumulative_EPM,Best_3_Years_Minutes,Draft Pick #,Draft Round,Draft Team,Draft Year,First_5_Years_Cumulative_EPM,First_5_Years_Minutes,Percentile_Best_3_Years_EPM,Percentile_Cumulative_EPM,...,Cumulative_EPM_2023,Minutes_2024,EPM_2024,Cumulative_EPM_2024,Total_Cumulative_EPM,Rank_Cumulative_EPM,First_5_Years_Cumulative_EPM,Rank_First_5_Years_EPM,Best_3_Years_Cumulative_EPM,Rank_Best_3_Years_EPM
0,0.000000,0.000,1,1,SAS,2023,8600.179398,2106.230,-3.448276,100.000000,...,0.0,2106.23,4.083210,8600.179398,8600.179398,1.0,8600.179398,1.0,0.000000,61.0
1,0.000000,0.000,2,1,CHO,2023,-3523.765930,2383.000,-5.172414,15.517241,...,0.0,2383.00,-1.478710,-3523.765930,-3523.765930,50.0,-3523.765930,50.0,0.000000,62.0
2,0.000000,0.000,3,1,POR,2023,-8878.799504,1765.390,-6.896552,1.724138,...,0.0,1765.39,-5.029370,-8878.799504,-8878.799504,58.0,-8878.799504,58.0,0.000000,63.0
3,0.000000,0.000,4,1,HOU,2023,2046.674204,1388.470,-8.620690,96.551724,...,0.0,1388.47,1.474050,2046.674204,2046.674204,3.0,2046.674204,3.0,0.000000,64.0
4,0.000000,0.000,5,1,DET,2023,177.735724,1582.870,-10.344828,94.827586,...,0.0,1582.87,0.112287,177.735724,177.735724,4.0,177.735724,4.0,0.000000,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,0.000000,0.000,56,2,DEN,2014,-1378.270761,456.984,-445.000000,53.333333,...,0.0,0.00,0.000000,0.000000,-1378.270761,29.0,-1378.270761,29.0,0.000000,328.0
592,0.000000,0.000,57,2,IND,2014,0.000000,0.000,-446.666667,-135.000000,...,0.0,0.00,0.000000,0.000000,0.000000,142.0,0.000000,145.0,0.000000,329.0
593,-2760.291793,1313.333,58,2,SAS,2014,-3171.399834,912.333,15.000000,35.000000,...,0.0,0.00,0.000000,0.000000,-4432.764146,40.0,-3171.399834,39.0,-2760.291793,52.0
594,0.000000,0.000,59,2,TOR,2014,0.000000,0.000,-448.333333,-136.666667,...,0.0,0.00,0.000000,0.000000,0.000000,143.0,0.000000,146.0,0.000000,330.0


In [11]:

# Save the modified dataframe to a new CSV file
output_file = 'data/Draft_Classes_with_EPM_all_years.csv'
draft_classes.to_csv(output_file, index=False)

output_file


'data/Draft_Classes_with_EPM_all_years.csv'